In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import shapiro, zscore

In [3]:
df = pd.read_csv("results/flask/results_flask_1.csv")

In [6]:
for col in df.columns:
    stat, p_value = shapiro(df[col])
    print(f"{col}: W-statistic={stat}, p-value={p_value}") if p_value > 0.05 else ''

CPU_FREQUENCY_0: W-statistic=1.0, p-value=1.0
CPU_FREQUENCY_1: W-statistic=1.0, p-value=1.0
CPU_FREQUENCY_2: W-statistic=1.0, p-value=1.0
CPU_FREQUENCY_3: W-statistic=1.0, p-value=1.0
CPU_FREQUENCY_4: W-statistic=1.0, p-value=1.0
CPU_FREQUENCY_5: W-statistic=1.0, p-value=1.0
CPU_FREQUENCY_6: W-statistic=1.0, p-value=1.0
CPU_FREQUENCY_7: W-statistic=1.0, p-value=1.0
DRAM_ENERGY (J): W-statistic=1.0, p-value=1.0
GPU0_MEMORY_TOTAL: W-statistic=1.0, p-value=1.0
TOTAL_MEMORY: W-statistic=1.0, p-value=1.0
TOTAL_SWAP: W-statistic=1.0, p-value=1.0


/home/sofiakonovalova/anaconda3/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
